<a href="https://colab.research.google.com/github/benjaminbrown038/Hugging-Face/blob/main/semantic_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>